In [6]:
import csv
import errno
import itertools
import os
import time

from bs4 import BeautifulSoup
import click
import dataset
import funcy as fy
import requests

from pyquery import PyQuery
import time
import traceback

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline

from datetime import datetime
import time

In [7]:
state_id = 0
gender = 0
number_of_results = 40000
headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'en-US,en;q=0.8',
        'Cache-Control': 'max-age=0',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Origin': 'http://registration.baa.org',
        'Referer': 'http://registration.baa.org/2016/cf/Public/iframe_ResultsSearch.cfm?mode=results',
    }
params = {
        'mode': 'results',
        'criteria': '',
        'StoredProcParamsOn': 'yes',
        'VarGenderID': gender,
        'VarBibNumber': '',
        'VarLastName': '',
        'VarFirstName': '',
        'VarStateID': state_id,
        'VarCountryOfResID': 0,
        'VarCountryOfCtzID': 0,
        'VarReportingSegID': 1,
        'VarAwardsDivID': 0,
        'VarQualClassID': 0,
        'VarCity': '',
        'VarTargetCount': number_of_results,
        'records': 25,
        'headerexists': 'Yes',
        'bordersize': 0,
        'bordercolor': '#ffffff',
        'rowcolorone': '#FFCC33',
        'rowcolortwo': '#FFCC33',
        'headercolor': '#ffffff',
        'headerfontface': 'Verdana,Arial,Helvetica,sans-serif',
        'headerfontcolor': '#004080',
        'headerfontsize': '12px',
        'fontface': 'Verdana,Arial,Helvetica,sans-serif',
        'fontcolor': '#000099',
        'fontsize': '10px',
        'linkfield': 'FormattedSortName',
        'linkurl': 'OpenDetailsWindow',
        'linkparams': 'RaceAppID',
        'queryname': 'SearchResults',
        'tablefields': 'FullBibNumber,FormattedSortName,AgeOnRaceDay,GenderCode,'
                       'City,StateAbbrev,CountryOfResAbbrev,CountryOfCtzAbbrev,'
                       'DisabilityGroup',
    }


In [8]:
results = []
for page_number, start in enumerate(itertools.count(1, 25)):
        # Don't hammer the server. Give it a sec between requests.
        time.sleep(1.0)

        click.echo('page %d of %d' % (page_number + 1, number_of_results/25))
        response = requests.post(
            'http://registration.baa.org/2016/cf/Public/iframe_ResultsSearch.cfm',
            headers=headers,
            params=params,
            data={'start': start, 'next': 'Next 25 Records'},
        )
        #peopleList = extract_data(response.content)
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find("table", attrs={"class": "tablegrid_table"})
        rows = table.findAll("tr")
        for row in rows:
            a = [t.text.strip() for t in row.findAll("td")][0:]
            #Don't store lines without data
            if len(a) > 0 and a != [''] and a !=['',''] and a != ['', '', '']: 
                results.append(a)

        # Only yield if there actually are results. Just found this random
        # tr_header thing in the HTML of the pages that have results, but not
        # empty results pages.
        if 'tr_header' in response.text:
            (page_number, response.text)
        else:
            assert 'Next 25 Records' not in response.text
            click.echo('  No results found.')
            break

        # No more pages!
        if 'Next 25 Records' not in response.text:
            break

page 1 of 1600
page 2 of 1600
page 3 of 1600
page 4 of 1600
page 5 of 1600
page 6 of 1600
page 7 of 1600
page 8 of 1600
page 9 of 1600
page 10 of 1600
page 11 of 1600
page 12 of 1600
page 13 of 1600
page 14 of 1600
page 15 of 1600
page 16 of 1600
page 17 of 1600
page 18 of 1600
page 19 of 1600
page 20 of 1600
page 21 of 1600
page 22 of 1600
page 23 of 1600
page 24 of 1600
page 25 of 1600
page 26 of 1600
page 27 of 1600
page 28 of 1600
page 29 of 1600
page 30 of 1600
page 31 of 1600
page 32 of 1600
page 33 of 1600
page 34 of 1600
page 35 of 1600
page 36 of 1600
page 37 of 1600
page 38 of 1600
page 39 of 1600
page 40 of 1600
page 41 of 1600
page 42 of 1600
page 43 of 1600
page 44 of 1600
page 45 of 1600
page 46 of 1600
page 47 of 1600
page 48 of 1600
page 49 of 1600
page 50 of 1600
page 51 of 1600
page 52 of 1600
page 53 of 1600
page 54 of 1600
page 55 of 1600
page 56 of 1600
page 57 of 1600
page 58 of 1600
page 59 of 1600
page 60 of 1600
page 61 of 1600
page 62 of 1600
page 63 of 1600
p

KeyboardInterrupt: 

In [9]:
data = []
for i, result in enumerate(results):
    if i%4 == 0:
        data.append(results[i] + results[i+1][1:])


In [10]:
columns = ['Bib', 'Name', 'Age', 'M/F', 'City', 'State', 'Country', 'Citizen', '', '5K', '10K', '15K', '20K', 'Half',
          '25K', '30K', '35K', '40K', 'Pace', 'Proj Time', 'Official Time', 'Overall', 'Gender', 'Division']

In [11]:
filename = 'marathon_results_2016_test.csv'

In [26]:
df = pd.DataFrame(data, columns = ['Bib', 'Name', 'Age', 'M/F', 'City', 'State', 'Country', 'Citizen', '', '5K', '10K', '15K', '20K', 'Half',
          '25K', '30K', '35K', '40K', 'Pace', 'Proj Time', 'Official Time', 'Overall', 'Gender', 'Division'])

In [27]:
df.to_csv(filename, index=False)

In [30]:
#pd.plotting.scatter_matrix(df)